# Assistants API Overview

The new [Assistants API](https://platform.openai.com/docs/assistants/overview) is a stateful evoltuion of our [Chat Completions API](https://platform.openai.com/docs/guides/text-generation/chat-completions-api) meant to simplify the creation of assistant-like experiences, and enable developer access to powerful tools like Code Interpreter and Knowledge Retreival.

## Chat Completions API vs Assistants API

The primitives of the **Chat Completions API** are `Messages`, on which you perform on `Completion` with a `Model` (`gpt-3.5-turbo`, `gpt-4`, etc). It is lightweight and powerful, but inherently stateless, which means you have to manage conversation state, tool definitions, retrieval documents, and code execution manually.

The primitives of the Assistants API are
- `Assistants`, which encapsulate instructions, tools, and (context) documents,
- `Threads`, which represent the state of a conversation, and
- `Runs`, which power the execution of an `Assistant` on a `Thread`, including multi-step tool use and regular responses.

We'll take a look at how these can be used to create powerful, stateful experiences!

## Python SDK Setup

> **Note**
> We've updated our Python SDK to add support for the Assistants API functions, so you'll need to update it to the latest version (`1.1.1` at time of writing).

In [ ]:
!pip install --upgrade openai

And make sure it's up to date by running:

In [ ]:
!pip show openai | grep Version

## Complete Example with Assistants API

### Assistants

The easiest way to get started with the Assistants API is through the [Assistants Playground](https://platform.openai.com/playground).

<img width="1728" alt="Screenshot 2023-11-08 at 3 28 51 PM" src="https://github.com/ibigio/shell-ai/assets/25421602/11aafc0c-6a03-4dd3-a8f9-ee2f749c30c8">

Let's begin by creating an assistant! We'll create a Math Tutor just like in our [docs](https://platform.openai.com/docs/assistants/overview).

<img width="1728" alt="Screenshot 2023-11-08 at 3 31 54 PM" src="https://github.com/ibigio/shell-ai/assets/25421602/f4012fde-875b-42b8-bfe1-21be189d7cfc">

You can view Assistants you've created in the [Assistants Dashboard](https://platform.openai.com/assistants).

<img width="1728" alt="Screenshot 2023-11-08 at 3 35 15 PM" src="https://github.com/ibigio/shell-ai/assets/25421602/2f0ac3c1-0e1a-4e12-8697-fa2047f9c740">

You can also create Assistants directly through the Assistants API, like so:

In [185]:
from openai import OpenAI
import json

client = OpenAI()

# Just a helper for more readable output
def show_json(obj):
    return json.loads(obj.json())

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_FzJSwsfIcBfSWWm8rSCwReI5',
 'created_at': 1699478811,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

Regardless of whether you create your Assistant through the Dashboard or with the API, you'll want to keep track of the Assistant ID. This is how you'll refer to your Assistant throughout Threads and Runs.

Next, we'll create a new Thread and add a message to it. This will hold the state of our conversation, so we don't have re-send the entire message history each time.


### Threads

In [186]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_HF1YO13gNDrN5rA0vxJ8txSN',
 'created_at': 1699478814,
 'metadata': {},
 'object': 'thread'}

In [187]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
show_json(message)

{'id': 'msg_6DrRvEvKfoKDI2J3uOHpUP0x',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1699478817,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_HF1YO13gNDrN5rA0vxJ8txSN'}

> **Note**
> Even though you're only sending new messages, you will still be charged for the tokens of the entire conversation history with each `Run`.

### Runs

Notice how the Thread we created is **not** associated with the Assistant we just created! Threads exist independently from Assistants, which may be different from what you'd expect if you've used ChatGPT (where a thread is tied to a model/GPT).

To get a completion from an Assistant for a given Thread, we must create a Run. Creating a Run will indicate to an Assistant it should look at the messages in the Thread and take action: either by adding a single response, or using tools. 

> **Note**
> Runs are key difference between the Assistants API and Chat Completions API! While in Chat Completions the model will only ever respond with a single message, in the Assistants API a Run may result in an Assistant using one or multiple tools, and potentially adding multiple messages to the Thread.

To get our Assistant to respond to the user, let's create the Run. As mentioned earlier, you must specify _both_ the Assistant _and_ the Thread.

In [188]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_Jc417uexRM9o0EMUT6A6pehG',
 'assistant_id': 'asst_FzJSwsfIcBfSWWm8rSCwReI5',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1699478821,
 'expires_at': 1699479421,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_HF1YO13gNDrN5rA0vxJ8txSN',
 'tools': []}

Unlike creating a completion in Chat Completions, creating a Run is an asynchronous operation. It will return immediately with the Run's metadata, which includes a `status` that will initially be set to `queued`. The `status` will be updated as the Assistant performs operations (like using tools and adding messages).

To know when the Assistant has completed processing, we can poll the Run in a loop. (We will soon be adding support for streaming.) While we here are only checking for `completed` and `finished`, in practice a `Run` may undergo a variety of status changes which you can choose to surface to the user. (These are called `Steps`, and will be covered later.)

In [ ]:
def wait_on_run(run):
    while not (run.status == "completed" or run.status == "failed"):
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
    return run

In [ ]:
run = wait_on_run(run)
show_json(run)

### Messages

Now that the Run has completed we list the Messages in the Thread to see what got added by the Assistant!

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

As you can see, Messages are ordered in reverse-chronological order – this was done so the results are always on the first `page` (since results can be paginated). Do keep a look out for this, since this is opposite to messages in the Chat Completions.

Now let's ask our Assistant to explain the result a bit further!

In [ ]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run)

# Get all the messages added after our last user message
messages = client.beta.threads.messages.list(thread_id=thread.id, order='asc', after=message.id)
show_json(messages)

This may feel like a lot of steps to get a response back, especially for this simple example. However, you'll soon see how we can add very powerful functionality to our Assistant without changing much code at all!

### Example

Let's take a look at how we could potentially put all of this together. Below is all the code you need to use an Assistant you've created.

TODO: free concurrency!

In [211]:
from openai import OpenAI

MATH_ASSISTANT_ID = "asst_ooPQXw1Jqx0DMH96PEmdvXrw"

client = OpenAI()

def get_response(thread):
    messages = client.beta.threads.messages.list(
        thread_id=thread.id, order="asc"
    )
    return [m.content[0].text.value for m in messages.data]

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )
    return run

In [212]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run

# Emulating concurrent user requests
thread1, run1 = create_thread_and_run("I need to solve the equation `3x + 11 = 14`. Can you help me?")
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?")
thread3, run3 = create_thread_and_run("I don't like math. What can I do?")

# Now all Runs are executing...

In [213]:
def wait_on_run(thread, run):
    while not (run.status == "completed" or run.status == "failed"):
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
    return run

# Wait for Run 1
run1 = wait_on_run(thread1, run1)
print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(thread2, run2)
print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(thread3, run3)
print(get_response(thread3))


['I need to solve the equation `3x + 11 = 14`. Can you help me?', 'Yes, first subtract 11 from both sides to get `3x = 3`, then divide both sides by 3 to get `x = 1`.']
['Could you explain linear algebra to me?', 'Linear algebra is the branch of mathematics concerning linear equations, linear functions, and their representations through matrices and vector spaces.']
["I don't like math. What can I do?", 'Explore different ways of learning math, such as through games, apps, or relating it to your interests, and consider a personal tutor who can tailor lessons to your learning style.']


What you may have noticed is that this code is actually not specific to our math Assistant at all... this code will work for any new Assistant you create! This is the power of the Assistants API.

## Code Interpreter

